In [7]:
import os
import tensorflow.compat.v1 as tf



model_checkpoint_dir = "/Users/vkapoor/Downloads/training_seed_05"
model_checkpoint = "model.ckpt_epoch89"
# Load the meta graph
graph = tf.Graph()
with graph.as_default():
    with tf.device('/cpu:0'):
        saver = tf.train.import_meta_graph(os.path.join(model_checkpoint_dir, model_checkpoint + '.meta'))


    
with tf.Session(graph=graph) as sess:
     # Get the problematic operation by its name
    strided_slice_op = graph.get_operation_by_name('tower_0/strided_slice')

    # Force the problematic operation to be placed on the CPU device
    with tf.device('/cpu:0'):
        sess.run(strided_slice_op) 
    # Restore the weights
    saver.restore(sess, os.path.join(model_checkpoint_dir, model_checkpoint))

      




InvalidArgumentError: Graph execution error:

Detected at node 'tower_0/strided_slice' defined at (most recent call last):
Node: 'tower_0/strided_slice'
Cannot assign a device for operation tower_0/strided_slice: {{node tower_0/strided_slice}} was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[tower_0/strided_slice]]